In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import findspark
findspark.init()

/tmp/ipykernel_14842/4280649393.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
from pyspark.sql import SparkSession

import os

# Set SPARK_LOCAL_IP to 127.0.0.1
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

# Initialize Spark session with a conservative memory allocation
spark = SparkSession.builder \
    .appName("USPS") \
    .config("spark.executor.memory", "8G") \
    .config("spark.driver.memory", "8G") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "4G") \
    .getOrCreate()

# Set the log level to ERROR to suppress WARN messages
spark.sparkContext.setLogLevel("ERROR")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/12 10:18:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("/home/pk/DAEN690/package_data/PKG_Origin_PcLevel.txt", header=True, sep="|")

In [4]:
df.show(3)

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
|88228305910|           001 |               37040|       1/16/2024|            1/16/2024|              15613|           G0|
|88180422391|           001 |               37090|       1/16/2024|            1/16/2024|              92247|           G0|
|88147885963|           055 |               37167|       1/11/2024|            1/11/2024|              33770|           PM|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
only showing top 3 rows



In [8]:
df.select('MailClassCode').distinct().show()

+-------------+
|MailClassCode|
+-------------+
|           PM|
|           BS|
|           FC|
|           EX|
|             |
|           SA|
|           BL|
|           IL|
|           G0|
|           LC|
|           RP|
|           CP|
|           S2|
|           IE|
|           G1|
|           BB|
|           UP|
|           DM|
+-------------+



In [13]:
df.count()

1115279

In [14]:
##### Local zipcodes to Nashville area ########
zips = ['37013','37027','37072','37076','37115','37138','37201','37203','37204','37205',
        '37206','37207','37208','37209','37210','37211','37212','37214','37215','37216',
        '37217','37218','37219','37220','37221','37027','37064','37067','37069','37135',
        '37014','37046','37062','37179','37174','37122','37121','37087','37090','37184',
        '37138','37075','37075','37066','37148','37070','37072','37048','37188','37086',
        '37167','37127','37218','37130']
# Filter DataFrame where zip_code exists in the list
df = df.filter(df.StartTheClockZipCode.isin(zips))
df = df.filter(df.StopTheClockZipCode.isin(zips))
df.count()

22794

In [15]:
df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTheClockZipCode: string (nullable = true)
 |-- StopTheClockDate: string (nullable = true)
 |-- ScheduledDeliveryDate: string (nullable = true)
 |-- StopTheClockZipCode: string (nullable = true)
 |-- MailClassCode: string (nullable = true)



In [16]:
from pyspark.sql.functions import to_date

date_fields = ["StopTheClockDate","ScheduledDeliveryDate"]

for field in date_fields:
    df = df.withColumn(field, to_date(df[field], "M/d/yyyy"))

# Show the updated DataFrame
df.show(3)
df.printSchema()

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
|88341629744|           490 |               37217|      2024-01-18|           2024-01-19|              37216|           BS|
|88387239947|           055 |               37211|      2024-01-20|           2024-01-20|              37216|           PM|
|88203064922|           001 |               37206|      2024-01-11|           2024-01-12|              37201|           G0|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+
only showing top 3 rows

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTh

In [17]:
from pyspark.sql.functions import col  # Import the col function

duplicate_records_with_counts = df.groupBy("MailPieceID") \
    .count() \
    .filter(col("count") > 1)

# Show if any duplicate records with their counts
duplicate_records_with_counts.show()

+-----------+-----+
|MailPieceID|count|
+-----------+-----+
+-----------+-----+



In [22]:
df2 = spark.read.parquet("/home/pk/DAEN690/Scans_processed.parquet")
print(df2.count())
df2.show(3)

44476
+-----------+-------------------+-------------------+------------------+--------------------+------------------+
|MailPieceID|  min_scan_datetime|  max_scan_datetime|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|
+-----------+-------------------+-------------------+------------------+--------------------+------------------+
|88244076233|2024-01-14 05:24:39|2024-01-25 05:40:53|                 3|                   1|             15840|
|88217514369|2024-01-14 04:27:22|2024-01-18 06:02:59|                 3|                   1|              5760|
|88298654737|2024-01-19 06:37:56|2024-01-23 04:13:54|                 3|                   1|              5760|
+-----------+-------------------+-------------------+------------------+--------------------+------------------+
only showing top 3 rows



In [23]:
# Join the two DataFrames on MailPieceID
joined_df = df.join(df2, "MailPieceID", "inner")
print(joined_df.count())
joined_df.show(3)

19131
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+------------------+--------------------+------------------+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|  min_scan_datetime|  max_scan_datetime|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+------------------+--------------------+------------------+
|88341629744|           490 |               37217|      2024-01-18|           2024-01-19|              37216|           BS|2024-01-18 03:14:46|2024-01-18 03:14:46|                 1|                   1|                 0|
|88387239947|           055 |               37211|      2024-01-20|           2024-01-20|             

In [24]:
joined_df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTheClockZipCode: string (nullable = true)
 |-- StopTheClockDate: date (nullable = true)
 |-- ScheduledDeliveryDate: date (nullable = true)
 |-- StopTheClockZipCode: string (nullable = true)
 |-- MailClassCode: string (nullable = true)
 |-- min_scan_datetime: timestamp (nullable = true)
 |-- max_scan_datetime: timestamp (nullable = true)
 |-- Distinct_zip_scans: long (nullable = true)
 |-- Distinct_event_scans: long (nullable = true)
 |-- time_delta_minutes: integer (nullable = true)



In [25]:
from pyspark.sql.functions import col, to_date

joined_df = joined_df.withColumn("min_date", to_date(col("min_scan_datetime")))
joined_df = joined_df.withColumn("max_date", to_date(col("max_scan_datetime")))


In [26]:
from pyspark.sql.functions import when, col, expr
joined_df = joined_df.withColumn("late", when(col("StopTheClockDate") > col("ScheduledDeliveryDate"), 1).otherwise(0))

In [27]:
joined_df.show(3)

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+------------------+--------------------+------------------+----------+----------+----+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|  min_scan_datetime|  max_scan_datetime|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|  min_date|  max_date|late|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+------------------+--------------------+------------------+----------+----------+----+
|88341629744|           490 |               37217|      2024-01-18|           2024-01-19|              37216|           BS|2024-01-18 03:14:46|2024-01-18 03:14:46|                 1|                   1|                 0|2024-01-18|2024-01-18|   0|


In [28]:
joined_df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTheClockZipCode: string (nullable = true)
 |-- StopTheClockDate: date (nullable = true)
 |-- ScheduledDeliveryDate: date (nullable = true)
 |-- StopTheClockZipCode: string (nullable = true)
 |-- MailClassCode: string (nullable = true)
 |-- min_scan_datetime: timestamp (nullable = true)
 |-- max_scan_datetime: timestamp (nullable = true)
 |-- Distinct_zip_scans: long (nullable = true)
 |-- Distinct_event_scans: long (nullable = true)
 |-- time_delta_minutes: integer (nullable = true)
 |-- min_date: date (nullable = true)
 |-- max_date: date (nullable = true)
 |-- late: integer (nullable = false)



In [29]:
joined_df.write.mode('overwrite').parquet("/home/pk/DAEN690/PKG_Origin_Piece_Scans.parquet")
print("done")

done


In [30]:
############### Combine Weathr Data ###############

In [31]:
df_PKG_O = spark.read.parquet("/home/pk/DAEN690/PKG_Origin_Piece_Scans.parquet")

In [32]:
df_PKG_O.show(3)
df_PKG_O.printSchema()

+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+------------------+--------------------+------------------+----------+----------+----+
|MailPieceID|ServiceTypeCode|StartTheClockZipCode|StopTheClockDate|ScheduledDeliveryDate|StopTheClockZipCode|MailClassCode|  min_scan_datetime|  max_scan_datetime|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|  min_date|  max_date|late|
+-----------+---------------+--------------------+----------------+---------------------+-------------------+-------------+-------------------+-------------------+------------------+--------------------+------------------+----------+----------+----+
|88240074381|           001 |               37205|      2024-01-12|           2024-01-16|              37205|           G0|2024-01-12 05:22:55|2024-01-12 05:22:55|                 1|                   1|                 0|2024-01-12|2024-01-12|   0|


In [33]:
from pyspark.sql.functions import min, max
min_date_Ori = df_PKG_O.agg(min("min_date")).first()[0]
max_date_Ori = df_PKG_O.agg(max("max_date")).first()[0]

# Show the minimum date
print("Minimum Date Ori:", min_date_Ori)
print("Max Date Ori:", max_date_Ori)
print(df_PKG_O.count())

Minimum Date Ori: 2023-11-08
Max Date Ori: 2024-05-11
19131


In [34]:
########## Join Weather data to USPS data #############

In [35]:
from pyspark.sql.functions import to_date, col
df_weather = spark.read.csv("weather_data.csv", header=True)

In [36]:
df_weather.printSchema()
df_weather.show(3)

root
 |-- Zip: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- TMIN: string (nullable = true)
 |-- TMAX: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- SNOW: string (nullable = true)

+-----+----------+----+----+----+----+
|  Zip|      Date|TMIN|TMAX|PRCP|SNOW|
+-----+----------+----+----+----+----+
|37010|2024-01-11|None|None| 0.0| 0.0|
|37010|2024-01-13|None|None|0.27|None|
|37010|2024-01-21|None|None| 0.0| 0.0|
+-----+----------+----+----+----+----+
only showing top 3 rows



In [37]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType, DateType

df_weather = df_weather.withColumn("Date", col("Date").cast(DateType()))
df_weather = df_weather.withColumn("TMIN", col("TMIN").cast(FloatType()))
df_weather = df_weather.withColumn("TMAX", col("TMAX").cast(FloatType()))
df_weather = df_weather.withColumn("PRCP", col("PRCP").cast(FloatType()))
df_weather = df_weather.withColumn("SNOW", col("SNOW").cast(FloatType()))

In [38]:
df_weather.printSchema()
df_weather.show(3)

root
 |-- Zip: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- TMIN: float (nullable = true)
 |-- TMAX: float (nullable = true)
 |-- PRCP: float (nullable = true)
 |-- SNOW: float (nullable = true)

+-----+----------+----+----+----+----+
|  Zip|      Date|TMIN|TMAX|PRCP|SNOW|
+-----+----------+----+----+----+----+
|37010|2024-01-11|NULL|NULL| 0.0| 0.0|
|37010|2024-01-13|NULL|NULL|0.27|NULL|
|37010|2024-01-21|NULL|NULL| 0.0| 0.0|
+-----+----------+----+----+----+----+
only showing top 3 rows



In [39]:
# Perform the join on Origin
df_joined = df_PKG_O.join(df_weather, (df_PKG_O["StartTheClockZipCode"] == df_weather["Zip"]) & (df_PKG_O["min_date"] == df_weather["Date"]), how="inner")

# List of columns to rename
columns_to_rename = ['Zip', 'Date', 'TMIN', 'TMAX', 'PRCP', 'SNOW']

# New column names with '_O' suffix
new_column_names = [f"{col}_O" for col in columns_to_rename]

# Rename columns using withColumnRenamed in a loop
for old_col, new_col in zip(columns_to_rename, new_column_names):
    df_joined = df_joined.withColumnRenamed(old_col, new_col)

df_joined.printSchema()
df_joined.show(3)
df_joined.count()

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTheClockZipCode: string (nullable = true)
 |-- StopTheClockDate: date (nullable = true)
 |-- ScheduledDeliveryDate: date (nullable = true)
 |-- StopTheClockZipCode: string (nullable = true)
 |-- MailClassCode: string (nullable = true)
 |-- min_scan_datetime: timestamp (nullable = true)
 |-- max_scan_datetime: timestamp (nullable = true)
 |-- Distinct_zip_scans: long (nullable = true)
 |-- Distinct_event_scans: long (nullable = true)
 |-- time_delta_minutes: integer (nullable = true)
 |-- min_date: date (nullable = true)
 |-- max_date: date (nullable = true)
 |-- late: integer (nullable = true)
 |-- Zip_O: string (nullable = true)
 |-- Date_O: date (nullable = true)
 |-- TMIN_O: float (nullable = true)
 |-- TMAX_O: float (nullable = true)
 |-- PRCP_O: float (nullable = true)
 |-- SNOW_O: float (nullable = true)

+-----------+---------------+--------------------+----------------+-----

16928

In [40]:
# Perform the join on Destination
df_joined = df_joined.join(df_weather, (df_joined["StopTheClockZipCode"] == df_weather["Zip"]) & (df_joined["max_date"] == df_weather["Date"]), how="inner")

# List of columns to rename
columns_to_rename = ['Zip', 'Date', 'TMIN', 'TMAX', 'PRCP', 'SNOW']

# New column names with '_O' suffix
new_column_names = [f"{col}_D" for col in columns_to_rename]

# Rename columns using withColumnRenamed in a loop
for old_col, new_col in zip(columns_to_rename, new_column_names):
    df_joined = df_joined.withColumnRenamed(old_col, new_col)

df_joined.printSchema()
df_joined.show(3)
df_joined.count()

root
 |-- MailPieceID: string (nullable = true)
 |-- ServiceTypeCode: string (nullable = true)
 |-- StartTheClockZipCode: string (nullable = true)
 |-- StopTheClockDate: date (nullable = true)
 |-- ScheduledDeliveryDate: date (nullable = true)
 |-- StopTheClockZipCode: string (nullable = true)
 |-- MailClassCode: string (nullable = true)
 |-- min_scan_datetime: timestamp (nullable = true)
 |-- max_scan_datetime: timestamp (nullable = true)
 |-- Distinct_zip_scans: long (nullable = true)
 |-- Distinct_event_scans: long (nullable = true)
 |-- time_delta_minutes: integer (nullable = true)
 |-- min_date: date (nullable = true)
 |-- max_date: date (nullable = true)
 |-- late: integer (nullable = true)
 |-- Zip_O: string (nullable = true)
 |-- Date_O: date (nullable = true)
 |-- TMIN_O: float (nullable = true)
 |-- TMAX_O: float (nullable = true)
 |-- PRCP_O: float (nullable = true)
 |-- SNOW_O: float (nullable = true)
 |-- Zip_D: string (nullable = true)
 |-- Date_D: date (nullable = true)


17503

In [41]:
# Convert Spark DataFrame to Pandas DataFrame
df_original = df_joined.toPandas()

/opt/spark/python/pyspark/sql/pandas/types.py:563: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not is_datetime64tz_dtype(pser.dtype):
/opt/spark/python/pyspark/sql/pandas/types.py:379: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):
/opt/spark/python/pyspark/sql/pandas/types.py:563: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if not is_datetime64tz_dtype(pser.dtype):
/opt/spark/python/pyspark/sql/pandas/types.py:379: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(s.dtype):


In [42]:
df_original.drop(columns=['ScheduledDeliveryDate','MailPieceID','StopTheClockDate',
                          'min_scan_datetime','max_scan_datetime','StartTheClockZipCode',
                          'StopTheClockZipCode','Date_O','Date_D','min_date','max_date','Distinct_zip_scans'], inplace=True)

In [44]:
##### Local zipcodes to Nashville area ########
zips = ['37013','37027','37072','37076','37115','37138','37201','37203','37204','37205',
        '37206','37207','37208','37209','37210','37211','37212','37214','37215','37216',
        '37217','37218','37219','37220','37221','37027','37064','37067','37069','37135',
        '37014','37046','37062','37179','37174','37122','37121','37087','37090','37184',
        '37138','37075','37075','37066','37148','37070','37072','37048','37188','37086',
        '37167','37127','37218','37130']

In [45]:
df_original = df_original[df_original['Zip_D'].isin(zips)]
df_original = df_original[df_original['Zip_O'].isin(zips)]

In [46]:
len(df_original.index) #17503

17503

In [47]:
# df.to_csv("Package_ML.csv", header=True)
df_original.to_parquet('Package_ML.parquet.gzip', compression='gzip') 

/home/pk/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [48]:
spark.stop()